<a href="https://colab.research.google.com/github/Krishna4123/amdl/blob/main/C2NCD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import zipfile
import os

zip_path = "/content/dataset.zip"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content")

print(os.listdir("/content"))


['.config', '.ipynb_checkpoints', 'dataset.zip', 'dataset', 'sample_data']


In [3]:
import os
import shutil
from sklearn.model_selection import train_test_split

root = "/content/dataset"   # <-- THIS IS YOUR CORRECT FOLDER

# Filter out directories like .ipynb_checkpoints
cats_dir = os.path.join(root, "cats")
dogs_dir = os.path.join(root, "dogs")
cats = [f for f in os.listdir(cats_dir) if os.path.isfile(os.path.join(cats_dir, f))]
dogs = [f for f in os.listdir(dogs_dir) if os.path.isfile(os.path.join(dogs_dir, f))]

# Create train/val directories
os.makedirs("/content/dataset_split/train/cats", exist_ok=True)
os.makedirs("/content/dataset_split/train/dogs", exist_ok=True)
os.makedirs("/content/dataset_split/val/cats", exist_ok=True)
os.makedirs("/content/dataset_split/val/dogs", exist_ok=True)

# Train/Val 80-20 split
train_cats, val_cats = train_test_split(cats, test_size=0.2, random_state=42)
train_dogs, val_dogs = train_test_split(dogs, test_size=0.2, random_state=42)

# Copy cat images
for img in train_cats:
    shutil.copy(os.path.join(root, "cats", img), "/content/dataset_split/train/cats")

for img in val_cats:
    shutil.copy(os.path.join(root, "cats", img), "/content/dataset_split/val/cats")

# Copy dog images
for img in train_dogs:
    shutil.copy(os.path.join(root, "dogs", img), "/content/dataset_split/train/dogs")

for img in val_dogs:
    shutil.copy(os.path.join(root, "dogs", img), "/content/dataset_split/val/dogs")

print("Done! Train/Val folders created in /content/dataset_split")


Done! Train/Val folders created in /content/dataset_split


In [ ]:
import os
print(os.listdir('/content/dataset'))


['.ipynb_checkpoints']


In [4]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

train_data = datasets.ImageFolder("/content/dataset_split/train", transform=transform)
val_data   = datasets.ImageFolder("/content/dataset_split/val", transform=transform)

train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_data, batch_size=16, shuffle=False)

print("Train:", len(train_data))
print("Val:", len(val_data))


Train: 96
Val: 24


In [8]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torchvision import models

# device = "cuda" if torch.cuda.is_available() else "cpu"
# print("Using device:", device)

# # Load pretrained MobileNetV2
# model = models.mobilenet_v2(weights='IMAGENET1K_V1')

# # Freeze feature extractor
# for param in model.features.parameters():
#     param.requires_grad = False

# # Replace classifier for 2 classes (cat, dog)
# model.classifier[1] = nn.Linear(model.last_channel, 2)

# model = model.to(device)


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()

        # CONV BLOCK 1
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)

        # CONV BLOCK 2
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)

        # CONV BLOCK 3
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(2, 2)

        # FC LAYERS (224 → 112 → 56 → 28)
        self.fc1 = nn.Linear(128 * 28 * 28, 256)
        self.fc2 = nn.Linear(256, 2)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))

        x = x.view(x.size(0), -1)  # flatten

        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x



In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

model = SimpleCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)



Using device: cpu


In [10]:
epochs = 10

for epoch in range(epochs):
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    print(f"Epoch [{epoch+1}/{epochs}]  "
          f"Loss: {running_loss/len(train_loader):.4f}  "
          f"Train Acc: {100*correct/total:.2f}%")


Epoch [1/10]  Loss: 1.9358  Train Acc: 47.92%
Epoch [2/10]  Loss: 0.6804  Train Acc: 53.12%
Epoch [3/10]  Loss: 0.6441  Train Acc: 67.71%
Epoch [4/10]  Loss: 0.5765  Train Acc: 66.67%
Epoch [5/10]  Loss: 0.5365  Train Acc: 66.67%
Epoch [6/10]  Loss: 0.4950  Train Acc: 72.92%
Epoch [7/10]  Loss: 0.4301  Train Acc: 81.25%
Epoch [8/10]  Loss: 0.2889  Train Acc: 87.50%
Epoch [9/10]  Loss: 0.2862  Train Acc: 88.54%
Epoch [10/10]  Loss: 0.2319  Train Acc: 92.71%


In [11]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = outputs.max(1)

        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

print(f"Validation Accuracy: {100*correct/total:.2f}%")



Validation Accuracy: 91.67%


In [ ]:
torch.save(model.state_dict(), "/content/mobilenet_dog_cat.pth")
print("Model saved!")


Model saved!


In [12]:
from PIL import Image

def predict(image_path):
    model.eval()
    image = Image.open(image_path).convert("RGB")
    img = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(img)
        _, pred = torch.max(output, 1)

    return "Dog" if pred.item() == 1 else "Cat"


In [13]:
print(predict("/content/lion.jpeg"))


Cat
